In [3]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

%matplotlib inline

plt.style.use('seaborn-bright')

import pickle
from IPython.display import display
%load_ext memory_profiler
%load_ext line_profiler

In [2]:
# not in notes (!)
df = pd.DataFrame(np.ones(50).reshape(10,5), 
                      index=pd.date_range('1/1/2016', periods=10, freq='H' ),
                      columns=list('ABCDE'))
df['size']=0
df['size'] = pd.Series([0,0,0,0,200,0,0,2000,0,0], index=df.index)
df.loc[:,"size_csum"] = df['size'].cumsum()

start = pd.to_datetime('1/1/1900')
df.loc[:,"size_astime"] = start + pd.to_timedelta(df.size_csum,'ms')
df.index = df.size_astime

display(df)
print(df.E.resample('1000ms',label='right').sum())

# timestamp_col = df.timestamp.resample('30min',label='right').last()

# pd.to_datetime(timestamp_col).dropna()

# after going around the circle, the workaround has to be applying an extra operation:
# `pd.to_datetime(timestamp_col).dropna()`

# df

,A,B,C,D,E,size,size_csum,size_astime
size_astime,,,,,,,,
1900-01-01 00:00:00.000,1.0,1.0,1.0,1.0,1.0,0,0,1900-01-01 00:00:00.000
1900-01-01 00:00:00.000,1.0,1.0,1.0,1.0,1.0,0,0,1900-01-01 00:00:00.000
1900-01-01 00:00:00.000,1.0,1.0,1.0,1.0,1.0,0,0,1900-01-01 00:00:00.000
1900-01-01 00:00:00.000,1.0,1.0,1.0,1.0,1.0,0,0,1900-01-01 00:00:00.000
1900-01-01 00:00:00.200,1.0,1.0,1.0,1.0,1.0,200,200,1900-01-01 00:00:00.200
1900-01-01 00:00:00.200,1.0,1.0,1.0,1.0,1.0,0,200,1900-01-01 00:00:00.200
1900-01-01 00:00:00.200,1.0,1.0,1.0,1.0,1.0,0,200,1900-01-01 00:00:00.200
1900-01-01 00:00:02.200,1.0,1.0,1.0,1.0,1.0,2000,2200,1900-01-01 00:00:02.200
1900-01-01 00:00:02.200,1.0,1.0,1.0,1.0,1.0,0,2200,1900-01-01 00:00:02.200


size_astime
1900-01-01 00:00:01    7.0
1900-01-01 00:00:02    NaN
1900-01-01 00:00:03    3.0
Freq: 1000L, Name: E, dtype: float64


In [3]:
# flattening history for autoregression
df = pd.DataFrame({'val': [10,40,30,10,11,12,10,11,12,19]}, index=pd.date_range('2016-01-01 19:00:00', periods=10, freq='H' ) )
display(df)
def myf(df):
    for i in range(1,2):
        df['val_t-' + str(i)] = df['val'].shift(i)

    for i in range(1,5):
        df['d_val_t+' + str(i)] = df['val'].shift(-i) - df['val']

    df[['d_val_t+3','d_val_t+4']].mean(axis=1)
    return df

df.groupby(pd.TimeGrouper('D')).apply(myf)

,val
2016-01-01 19:00:00,10
2016-01-01 20:00:00,40
2016-01-01 21:00:00,30
2016-01-01 22:00:00,10
2016-01-01 23:00:00,11
2016-01-02 00:00:00,12
2016-01-02 01:00:00,10
2016-01-02 02:00:00,11
2016-01-02 03:00:00,12
2016-01-02 04:00:00,19


,val,val_t-1,d_val_t+1,d_val_t+2,d_val_t+3,d_val_t+4
2016-01-01 19:00:00,10,NaN,30.0,20.0,0.0,1.0
2016-01-01 20:00:00,40,10.0,-10.0,-30.0,-29.0,NaN
2016-01-01 21:00:00,30,40.0,-20.0,-19.0,NaN,NaN
2016-01-01 22:00:00,10,30.0,1.0,NaN,NaN,NaN
2016-01-01 23:00:00,11,10.0,NaN,NaN,NaN,NaN
2016-01-02 00:00:00,12,NaN,-2.0,-1.0,0.0,7.0
2016-01-02 01:00:00,10,12.0,1.0,2.0,9.0,NaN
2016-01-02 02:00:00,11,10.0,1.0,8.0,NaN,NaN
2016-01-02 03:00:00,12,11.0,7.0,NaN,NaN,NaN
2016-01-02 04:00:00,19,12.0,NaN,NaN,NaN,NaN


In [4]:
X_train = [[1,2,2],[3,4,4]]
y_train = [100,200]

# ElasticNet

enet = ElasticNet(alpha=0.1, l1_ratio=0.7)

y_pred_enet = enet.fit(X_train, y_train) # .predict(X_test)
# r2_score_enet = r2_score(y_test, y_pred_enet)
# print(enet)
# print("r^2 on test data : %f" % r2_score_enet)

plt.plot(enet.coef_, label='Elastic net coefficients')
# plt.plot(lasso.coef_, label='Lasso coefficients')
# plt.plot(coef, '--', label='original coefficients')
# plt.legend(loc='best')
# plt.title("Lasso R^2: %f, Elastic Net R^2: %f"
#           % (r2_score_lasso, r2_score_enet))
# plt.show()
 

NameError: name 'ElasticNet' is not defined

In [51]:
data = pd.DataFrame({'val': [10,40,30,10,11,12,10,11,12,19]}, index=pd.date_range('2016-01-01 19:00:00', periods=10, freq='H' ) )

class Myc(object):
    def __init__(self, df):
        self.df_raw = df
        
        self._df = self.df_raw
        self._start_time = self.df_raw.index[0]
        self._end_time   = self.df_raw.index[-1]

    def display(self):
        display(self.df)

    @property
    def start_time(self):
        return self._start_time
    
    @start_time.setter
    def start_time(self, datetime):
        self.df = self.df_raw[datetime: self.end_time]
        print 'self.df is updated'
        self._start_time = datetime
        
    @property
    def end_time(self):
        return self._end_time
    
    @end_time.setter
    def end_time(self, datetime):
        self.df = self.df_raw[datetime: self.end_time]
        print 'self.df is updated'
        self._end_time = datetime

 
        
myobj = Myc(data)
# myobj.display()
myobj.end_time = '2016-01-02 02:00:00'
print myobj.start_time

myobj.start_time = '2016-01-02 02:22:22'
print myobj.start_time 
print myobj.end_time 

myobj.display()


self.df is updated
2016-01-01 19:00:00
self.df is updated
2016-01-02 02:22:22
2016-01-02 02:00:00


,val


In [6]:
cos_similarity_outsample_array = np.empty(35+1)

cos_similarity_outsample_array[:] = np.NaN

cos_similarity_outsample_array

array([ nan,  nan,  nan,  nan,  nan,  nan,  nan,  nan,  nan,  nan,  nan,
        nan,  nan,  nan,  nan,  nan,  nan,  nan,  nan,  nan,  nan,  nan,
        nan,  nan,  nan,  nan,  nan,  nan,  nan,  nan,  nan,  nan,  nan,
        nan,  nan,  nan])

In [15]:
coef_array = np.empty((3,3))
coef_array[:] = np.NaN
coef_array[1,:] = [1,2,3]
coef_array

array([[ nan,  nan,  nan],
       [  1.,   2.,   3.],
       [ nan,  nan,  nan]])